In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import csv
import random
import cv2

In [ ]:
#Segregating the dataset.
path = "/kaggle/input/satellite-images-of-water-bodies/Water Bodies Dataset"
img_path = os.path.join(path, "Images")
img_list = os.listdir(img_path)
tc = int(len(img_list) * 0.8)
train_img = img_list[:tc]
test_img = img_list[tc:]
train_img[0]

In [ ]:
class WaterBodies():
    def __init__(self, path, img_list, transform = None):
        self.path = path
        self.img_list = img_list
        self.transform = transform
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = os.path.join(path, "Images")
        mask_path = os.path.join(path, "Masks")
        satImgPath = os.path.join(img_path, self.img_list[idx])
        maskImgPath = os.path.join(mask_path, self.img_list[idx])
        satImg = Image.open(satImgPath)
        maskImg = Image.open(maskImgPath)
        
        if self.transform:
            satImg = self.transform(satImg)
            maskImg = self.transform(maskImg)
            
        return satImg, maskImg

In [ ]:
img_transform = transforms.Compose([
    transforms.Resize((550, 550)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [ ]:
satTrain = WaterBodies(path, train_img, transform=img_transform)
satTest = WaterBodies(path, test_img, transform=img_transform)

In [ ]:
train_loader = DataLoader(dataset=satTrain, batch_size = 16, shuffle=True)
test_loader = DataLoader(dataset=satTest, batch_size = 16, shuffle=False)

In [ ]:
#Visualizing dataset
import matplotlib.pyplot as plt

img, msk = satTrain[0]
plt.imshow(img.numpy().transpose((1, 2, 0)))
# plt.imshow(msk.numpy().transpose((1, 2, 0)))
plt.axis('off')
plt.show()

In [ ]:
def convLayer(in_channels, out_channels):
    conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size = 3),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size = 3),
        nn.ReLU(inplace=True)
    )
    return conv

In [ ]:
def crop_tensor(source, target):
    _,_,h,w = target.shape
    source = transforms.CenterCrop([h, w])(source)
    return source

In [ ]:
def concat_tensor(target, skip):
    target = torch.cat([target, skip], axis=1)
    return target

In [ ]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.conv1 = convLayer(3, 64)
        self.maxPoolLayer = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = convLayer(64, 128)
        self.conv3 = convLayer(128, 256) 
        self.conv4 = convLayer(256, 512)
        self.conv5 = convLayer(512, 1024)
        self.conv6 = convLayer(1024, 512)
        self.conv7 = convLayer(512, 256)
        self.conv8 = convLayer(256, 128) 
        self.conv9 = convLayer(128, 64)
        self.upConv1 = nn.ConvTranspose2d(1024, 512, 2)
        self.upConv2 = nn.ConvTranspose2d(512, 256, 2)
        self.upConv3 = nn.ConvTranspose2d(256, 128, 2)
        self.upConv4 = nn.ConvTranspose2d(128, 64, 2)
        self.upConv5 = nn.ConvTranspose2d(64, 3, 2)
    
    def forward(self, image):
        x1 = self.conv1(image) 
        x2 = self.maxPoolLayer(x1)
        x3 = self.conv2(x2) 
        x4 = self.maxPoolLayer(x3)
        x5 = self.conv3(x4) 
        x6 = self.maxPoolLayer(x5)
        x7 = self.conv4(x6) 
        x8 = self.maxPoolLayer(x7)
        x9 = self.conv5(x8)
        x = self.upConv1(x9)
        y = concat_tensor(x, crop_tensor(x7, x))
        x = self.conv6(y)
        x = self.upConv2(x)
        y = concat_tensor(x, crop_tensor(x5, x))
        x = self.conv7(y)
        x = self.upConv3(x)
        y = concat_tensor(x, crop_tensor(x3, x))
        x = self.conv8(y)
        x = self.upConv4(x)
        y = concat_tensor(x, crop_tensor(x1, x))
        x = self.conv9(y)
        x = self.upConv5(x)

        return x

In [ ]:
#Checking the Model

x = torch.randn(2, 3, 527, 527)
model = UNet()
output = model(x)
print(model)

In [ ]:
beta = 0.99
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001, momentum=beta)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
num_ep = 30
for ep in range(num_ep):
    for img, msk in train_loader:
        img = img.to(device)
        msk = msk.to(device)
        
        optimizer.zero_grad()
        output = model(img)
        print(output.size(), msk.size())
        loss = criterion(output, msk)
        
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(output.data, 1)
        total_train += msk.nelement()
        correct_train += predicted.eq(msk.data).sum().item()
        train_accuracy = 100 * correct_train / total_train
        
        print(f'Epoch: [{ep + 1} / {num_ep}], Loss: {loss.item():.4f}')

In [ ]:
def pixel_accuracy(output, mask):
    with torch.no_grad():
        output = torch.argmax(F.softmax(output, dim=1), dim=1)
        correct = torch.eq(output, mask).int()
        accuracy = float(correct.sum()) / float(correct.numel())
    return accuracy

In [ ]:
def mIoU(pred_mask, mask, smooth=1e-10, n_classes=2):
    with torch.no_grad():
        pred_mask = nn.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union +smooth)
                iou_per_class.append(iou)
        return np.nanmean(iou_per_class)

In [ ]:
#Testing the Image Segmentation Models

model.eval()
miou_list = []
pix_acc = []
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
for img, msk in test_loader:
    with torch.no_grad():
        img = img.to(device)
        msk = msk.to(device)
        img.unsqueeze(0)
        msk.unsqueeze(0)
        output = model(img)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
        
        
        #PixelAccuracy
        pred, sc = pixel_accuracy(model, img, msk)
        pix_acc.append(sc)
        
        #MIOU
        pred, sc = mIOU(model, img, msk)
        miou_list.append(sc)

        
    
    
    